In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division
%matplotlib inline 

In [ ]:
mDIR ='/home/lindb/teakettle/postmoran/nuclear'
if not op.exists(mDIR):
    mkdir(mDIR)

# for all teakettle - nuclear only

### Put into R
```R
source('~/imports.R')
library(PopGenReport)
library(adegenet)

moran     <- read.csv('~/Desktop/moran_infile_final_jan_wconecounts.txt',sep='\t')
simpmoran <- moran[seq(1,nrow(moran),2),] # for spatial stuff

### name some stuff
chloro <- c('pc10','pt71936','pt87268')
nuc    <- c('rps12','rps2','rps39','rps50')
loci   <- c(nuc,chloro)

### make a blank df to fill in later
popgen <- matrix(0,ncol=len(nuc)+1,nrow=luni(moran$samp))
popgen <- data.frame(popgen)
colnames(popgen) <- c('pop',nuc)
rownames(popgen) <- uni(moran$samp)
head(popgen)

getgt = function(type,df,locus){
    if (type == "nuc"){
        one = df[1,locus]
        two = df[1,sprintf("%s_2",locus)]
        combined = sprintf("%s:%s",one,two)
    } else {
        combined = sprintf("%s:%s",df[1,locus],df[1,locus])
    }
    return(combined)
}

# make function to make df
getdf = function(treatment='tx'){
    if (!treatment %in% c('tx','plot','tfk')){
        stop(cat(sprintf("treatment %s not in %s",treatment,c('tx','plot','single'))))
    }
    # make blank df
    popgen <- matrix(0,ncol=len(nuc)+1,nrow=luni(moran$samp))
    popgen <- data.frame(popgen)
    colnames(popgen) <- c('pop',nuc)
    rownames(popgen) <- uni(moran$samp)

    # fill in blank df
    for (samp in uni(moran$samp)){
        df <- moran[which(moran$samp == samp),]

        plot = strsplit(samp,split="_")[[1]][1]
        tx   = substr(plot,1,2)
        if (treatment=='tx'){
            popgen[samp,'pop'] <- tx
        } else if (treatment == 'plot'){
            popgen[samp,'pop'] <- plot
        } else {
            popgen[samp,'pop'] <- 'teakettle'
        }
        for (locus in nuc){
            type = NULL
            if (locus %in% nuc){
                type = 'nuc'
            } else {
                type = 'chloro'}
            if (df[1,locus] != 0){
                popgen[samp,locus] = getgt(type,df,locus)
            } else if (df[2,locus] != 0) {
                popgen[samp,locus] = getgt(type,df,locus)
            } else {
                popgen[samp,locus] = NA}}}
    return(popgen)}

### make dfs with either plot or treatment
popgen.tfk  <- getdf(treatment='tfk')  # one single pop
head(popgen.tfk)
# df with only loci as columns
pg.tfk <- popgen.tfk[,names(popgen.tfk[2:len(names(popgen.tfk))])]
head(pg.tfk)

### convert to class(genind)
tfkpop  <- popgen.tfk$pop
gentfk  <- df2genind(pg.tfk,ploidy=2,sep=":",pop=tfkpop)

### calc gen dist
gd <- gd.smouse(gentfk)
gdmat <- as.matrix(gd)
dim(gdmat)
gdmat[1:5,1:5]

### get euclidian distance
distmat <- readRDS('~/fragment/postmoran/final/distmat.RDS')
dmat <- distmat[which(rownames(distmat) %in% rownames(gdmat)),which(colnames(distmat) %in% colnames(gdmat))]
all(rownames(dmat) == rownames(gdmat))
all(colnames(dmat) == colnames(gdmat))
luni(rownames(dmat)) == nrow(gdmat)
luni(rownames(gdmat)) == nrow(dmat)
all(dim(dmat) == dim(gdmat))

### get spatial autocorrelation
bins = ceiling(.max(dmat)/10) # = 173
sauto <- spautocor(gdmat,dmat,bins = 173)
range(sauto$r)

### save environment
cd('~/fragment/postmoran/all_tfk-nuclear')
rds()
```

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/nuclear/final'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
# copy from desktop to godel
# scp ~/fragment/postmoran/all_tfk-nuclear/*RDS godel:/home/lindb/teakettle/postmoran/nuclear/final

### multiprocess null confidence intervals for spatial autocorrelation

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/nuclear/null/rfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/nuclear/null/shfiles'
if not op.exists(DIR):
    mkdir(DIR)

In [ ]:
if not op.exists('/home/lindb/teakettle/postmoran/nuclear/null/outfiles/'):
    mkdir('/home/lindb/teakettle/postmoran/nuclear/null/outfiles/')

In [ ]:
for i in range(999):
    text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/nuclear/final')

# load dependencies
getrds()

spautocor <- function (gen.m, eucl.m, shuffle = FALSE, bins = 10) 
{
    gd <- gen.m
    ed <- eucl.m
    if (shuffle == TRUE) {
        gdd <- as.dist(gd)
        gdsample <- sample(1:length(gdd), length(gdd))
        gd[lower.tri(gd)] <- gdd[gdsample]
        gd[upper.tri(gd)] <- gdd[gdsample]
        diag(gd) <- 0
    }
    cdmat <- function(gd) {
        dimen <- nrow(gd)
        sgd <- sum(gd, na.rm = TRUE)
        cscd <- matrix(colSums(gd, na.rm = TRUE), dimen, dimen)
        rscd <- matrix(rowSums(gd, na.rm = TRUE), dimen, dimen, 
            byrow = TRUE)
        cd <- 0.5 * (-gd + 1/dimen * (cscd + rscd) - 1/dimen^2 * 
            (sgd))
        cd
    }
    cd <- cdmat(gd)
    ed[upper.tri(ed)] <- NA
    diag(ed) <- NA
    r <- NA
    distance <- NA
    N <- NA
    steps <- signif(diff(range(ed, na.rm = TRUE))/bins, 4)
    for (d in 1:bins) {
        index <- which(ed <= d * steps & ed > (d - 1) * steps, 
            arr.ind = TRUE)
        cx <- sum(cd[index])
        cxii <- sum(diag(cd)[index[, 1]])
        cxjj <- sum(diag(cd)[index[, 2]])
        r[d] <- 2 * cx/(cxii + cxjj)
        distance[d] <- steps * d
        N[d] <- length(index)
    }
    if (shuffle == FALSE) 
        res <- data.frame(bin = distance, N = N, r = r)
    else res <- data.frame(r = r)
    res
}

# get null value
shuff <- spautocor(gdmat,dmat,bins=bins,shuffle=T)
filE = sprintf('/home/lindb/teakettle/postmoran/nuclear/null/outfiles/shuff_%%s.RDS',sprintf('%%03d',%s))
saveRDS(shuff,filE)

''' % (i)
    filE = '/home/lindb/teakettle/postmoran/nuclear/null/rfiles/null_%s.R' % str(i).zfill(3)
    with open(filE,'w') as o:
        o.write("%s" % text)
    o.close()

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/nuclear/null/rfiles/')
math.ceil(len(files)/195)

In [ ]:
files = fs('/home/lindb/teakettle/postmoran/nuclear/null/rfiles/')

fcount  = 0
shcount = 0
rcount  = 0
for f in files:
    num = op.basename(f).split(".R")[0].split("_")[1]
    if rcount == 0:
        text = '''#!/bin/bash
#$ -N shuff%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -cwd

cd %s
R --no-save < %s
''' % (num,
       op.dirname(f),
       f)
    else:
        newtext = '''
cd %s
R --no-save < %s
''' % (op.dirname(f),
       f)
        text = text + newtext
    fcount += 1
    rcount += 1
    
    if rcount == 6 or fcount == len(files):
        filE = '/home/lindb/teakettle/postmoran/nuclear/null/shfiles/shuff_%s.sh' % str(shcount).zfill(3)
        with open(filE,'w') as o:
            o.write("%s" % text)
        rcount = 0
        shcount += 1

In [ ]:
# qsub sh files
# !cd /home/lindb/teakettle/postmoran/nuclear/null/shfiles
# !ls *sh | parallel qsub {}

### combine null runs

In [ ]:
DIR = '/home/lindb/teakettle/postmoran/nuclear/null/outfiles/'
text = '''
source('~/imports.R')
cd('/home/lindb/teakettle/postmoran/nuclear/')

sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')

# get a list of all of the null runs
DIR = '/home/lindb/teakettle/postmoran/nuclear/null/outfiles/'
R <- list()
for (f in list.files(DIR,full.names=T,pattern='.RDS')){
    r <- readRDS(f)
    R <- append(R,list(r))
    names(R)[len(R)] <- as.character(len(R))
}
r <- data.frame(sauto$r)
names(r) <- 'r'
R <- append(R,list(r))
names(R)[1000] <- as.character(1000)

# get a list for each distance class, appending values from 1000 null runs
Rm <- list()
for (i in 1:173){
    print(i)
    lst <- c()
    for (j in 1:len(R)){
        lst <- c(lst, R[[as.character(j)]][i,'r'])
    }
    Rm <- append(Rm,list(lst))
}

# get 2.5th and 97.5th quantiles
null <- list()
for (i in 1:len(Rm)){
    low = sort(Rm[[i]])[25]
    high = sort(Rm[[i]])[975]
    null <- append(null,list(c(low,high)))
}


filE = fname(getwd(),'nuclear_null.RDS')
saveRDS(null,filE)
filE = fname(getwd(),'nuclear_R.RDS')
saveRDS(R,filE)
filE = fname(getwd(),'nuclear_Rm.RDS')
saveRDS(Rm,filE)
'''
filE = '/home/lindb/teakettle/postmoran/nuclear/combine_R_vector_nuclear_geno.R'
with open(filE,'w') as o:
    o.write("%s" % text)
o.close()

In [ ]:
#! cd /home/lindb/teakettle/postmoran/nuclear/
#! R --no-save < /home/lindb/teakettle/postmoran/nuclear/combine_R_vector.R

In [ ]:
### plot spatial autocorrelation
### put into R
```R
source('~/imports.R')
cd('~/Downloads/')

### load Rm, R, and sauto
# null <- readRDS('/home/lindb/teakettle/postmoran/nuclear/nuclear_null.RDS')
# sauto <- readRDS('/home/lindb/teakettle/postmoran/nuclear/final/sauto.RDS')

getrds()
null <- nuclear_null

# get quantiles for each distance class
lows <- c()
his <- c()
for (i in 1:len(null)){
	lows <- c(lows,null[[i]][1])
	his <- c(his,null[[i]][2])
}

plot(sauto$bin,sauto$r,type="l",xlab="Distance (meters)",ylab="r",ylim=c(-0.13,0.13),xaxt="n")
xtick <- seq(0,max(sauto$bin),100)
axis(side=1,at=xtick)
polygon(c(rev(sauto$bin),sauto$bin),c(rev(his),lows),col='grey75',border=FALSE)
lines(sauto$bin,sauto$r,lwd=2)
lines(sauto$bin,lows,col="red",lty=2)
lines(sauto$bin,his,col="red",lty=2)
```

In [ ]:
# !scp ~/fragment/postmoran/all_tfk_nuclear.pdf godel:/home/lindb/teakettle/postmoran/figs

In [ ]:
from wand.image import Image as WImage
img = WImage(filename='/home/lindb/teakettle/postmoran/figs/all_tfk_nuclear.pdf')
img